In [ ]:
import laspy
import numpy as np
import scipy as sp
from tqdm import tqdm

import raster

In [ ]:
# Read the point cloud.
with laspy.open("lidar.laz") as src:
    data = src.read()

In [ ]:
# Compute the bounding box of the point cloud.
bbox = [*data.header.mins[:2], *data.header.maxs[:2]]

### Rasterization

In [ ]:
RESOLUTION = 0.25
ATTRIBUTE = data["Reflectance"].scaled_array()

In [ ]:
# Initialize the output raster.
out = raster.Raster(0.25, bbox=bbox)

# Compute the raster cells.
rows, cols = np.mgrid[0:out.height, 0:out.width]

cells_x = bbox[0] + (cols + 0.5) * RESOLUTION
cells_y = bbox[3] - (rows + 0.5) * RESOLUTION
cells = np.vstack(
    [cells_x.ravel(), cells_y.ravel()]
).transpose()

cells

In [ ]:
# Initialize the index.
index = sp.spatial.KDTree(np.vstack([data.x, data.y]).transpose())

In [ ]:
# Compute the cell neighbors.
neighbors = index.query_ball_point(cells, r=RESOLUTION, workers=-1, return_sorted=False)

In [ ]:
# Compute the distances to the cell neighbors.
# TODO: Cache the neighbor and distance queries.
# TODO: Add multithreading.
distances = np.empty_like(neighbors)
for i, (cell, neighbor) in enumerate(zip(cells, neighbors)):
    distances[i] = sp.spatial.distance.cdist(cell[None, :], index.data[neighbor]).ravel()

In [ ]:
# Interpolate the attribute value at the raster cells.
# TODO: Add multithreading.
field = np.full_like(neighbors, fill_value=np.nan, dtype=np.float32)
for i, (neighbor, distance) in tqdm(enumerate(zip(neighbors, distances)),
                                    desc="Rasterization",
                                    total=len(neighbors),
                                    unit="cells"):
    if not neighbor:
        # The cell is empty.
        continue
    if np.any(distance == 0):
        # The cell is a member of the point cloud.
        field[i] = ATTRIBUTE[neighbor[np.argsort(distance)[0]]]
    else:
        # IDW
        field[i] = np.average(ATTRIBUTE[neighbor], weights=distance ** -2)

# Overwrite the raster data.
out._data = field.reshape([out.height, out.width])
out.save("lidar.refl.tif")

### Normals

In [ ]:
# Compute the six nearest neighbors of each point.
# TODO: Explain why ``k=7``.
neighbors = index.query(index.data, k=6 + 1, workers=-1)[1]

In [ ]:
# Fit a plane to the local neighborhood of each point.
elevation = data["z"].scaled_array()

positive = np.array([0, 0, 1])
normals = np.full([index.n, 3], np.nan)
for i, neighbor in tqdm(
        enumerate(neighbors), 
        desc="Normal Computation", 
        total=len(neighbors), 
        unit="points"
):
    pts = np.hstack([index.data[neighbor], elevation[neighbor][:, None]])

    # NOTE: The final rotation matrix is returned in its Hermitian form,
    n = np.linalg.svd(
        pts - np.mean(pts, axis=0)
    )[-1][-1]
    if np.dot(n, positive) < 0:
        # The normal is pointing to the ground.
        n *= -1

    normals[i] = n

normals

In [ ]:
np.save("lidar.norm", normals)

### Radiometric Calibration

In [ ]:
normals = np.load("lidar.norms.npy")

In [ ]:
# Get the scan angle.
# TODO: Explain the scan angle conversion.
# https://github.com/ASPRSorg/LAS/issues/41
scan_angle = np.radians(data.scan_angle * 0.006)
scan_angle

In [ ]:
# Compute the refracted laser beam vectors.
# NOTE: This operation is the result of rotating the unit vector in the direction of the z-axis by the scan angle around the y-axis.
refracted_beam = np.vstack([np.sin(scan_angle), np.zeros(len(scan_angle)), np.cos(scan_angle)]).transpose()
refracted_beam

In [ ]:
incidence_angle = np.zeros(len(refracted_beam))
for i, (beam, n) in enumerate(zip(refracted_beam, normals)):
    # NOTE: The beam vectors may not be normalized due to floating-point rounding errors, and thus the complete trigonometric calculation is necessary.
    incidence_angle[i] = np.dot(beam, n) / np.linalg.norm(beam)

incidence_angle

In [ ]:
# FIXME: The 

In [ ]:
import matplotlib.pyplot as plt

plt.hist(incidence_angle)